In [1]:
import torch
flag = torch.cuda.is_available()
print(flag)

if flag:
    print(torch.cuda.get_device_name(0))
    print(torch.rand(3,3).cuda()) 

False


In [29]:
import sys
import os

# 确定项目根目录（假设当前工作目录是项目的根目录）
project_root = os.path.abspath("..")
sys.path.insert(0, project_root)

import pickle
from ai import ai_battle, MinimaxAI, MCTSAI
from game.chess_game import ChessGame
from CelestialVault.instances import ExampleThreadManager


class TrainDataThread(ExampleThreadManager):
    def get_args(self, obj: object):
        return (mcts_ai_0, mcts_ai_0, ChessGame((5, 5), 2), False)
    
    def process_result(self):
        all_training_data = []
        result_dict = self.get_result_dict()
        for d in result_dict:
            over_game = result_dict[d]
            history_board = over_game.history_board
            history_move = over_game.history_move
            for step in history_board:
                if step+1 not in history_board.keys():
                    continue
                board = self.process_board(history_board, step)
                all_training_data.append((board, history_move[step+1]))
        return all_training_data
    
    def process_board(self, history_board, step):
        color = 1 if step % 2 == 0 else -1
        return [[j + [color] for j in i] for i in history_board[step]]



minimax_ai = MinimaxAI(3)
mcts_ai_0 = MCTSAI(1000, flag=True)
mcts_ai_1 = MCTSAI(1000, flag=False)

train_data_threader = TrainDataThread(
            ai_battle,
            thread_num=10,
            tqdm_desc='trainDataProcess',
            show_progress=True)


In [30]:

from time import strftime, localtime


def train_data(train_num):
    train_data_threader.start(range(train_num), "serial")
    train_data_threader.handle_error()
    all_training_data = train_data_threader.process_result()

    data_size = len(all_training_data)
    now_time = strftime("%m-%d-%H", localtime())
    pickle.dump(all_training_data, open(f"train_data/all_training_data({now_time})({data_size}).pkl", "wb"))

    return all_training_data

all_training_data = train_data(100)

all_training_data[0]

trainDataProcess: 100%|██████████| 10/10 [08:08<00:00, 48.87s/it]


([[[0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1]],
  [[0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1]],
  [[0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1]],
  [[0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1]],
  [[0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1], [0, 0, 1]]],
 (3, 1))

In [6]:
import torch
from torch.utils.data import DataLoader, Dataset

class ChessDataset(Dataset):
    def __init__(self, data):
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        board_state, move = self.data[idx]
        board_state = torch.tensor(board_state, dtype=torch.float32)
        move = move[0] * 5 + move[1]
        return board_state, move

dataset = ChessDataset(all_training_data)

dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [7]:

import torch.nn as nn
import torch.optim as optim
from ai.deeplearning import ChessModel, DeepLearningAI

# 设置CuDNN选项
torch.backends.cudnn.benchmark = True

model = ChessModel().cuda() # 初始化模型，并将其移动到GPU上
criterion = nn.CrossEntropyLoss() # 定义交叉熵损失函数
optimizer = optim.Adam(model.parameters(), lr=0.001) # 定义Adam优化器

# 训练循环
num_epochs = 10  # 训练10个epoch
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data
        # 调整输入的维度，并将其移动到GPU上
        # inputs 的原始形状是 (batch_size, height, width, channels)，也就是 (32, 5, 5, 3)
        # inputs.permute(0, 3, 1, 2) 会将 inputs 的维度从 (32, 5, 5, 3) 转换为 (32, 3, 5, 5)
        inputs = inputs.permute(0, 3, 1, 2).cuda()  # (batch_size, channels, height, width)
        labels = labels.cuda()

        # 清零梯度
        optimizer.zero_grad()

        # 前向传播
        outputs = model(inputs)
        # 计算损失
        loss = criterion(outputs, labels)
        # 反向传播
        loss.backward()
        # 更新参数
        optimizer.step()

        # 累积损失
        running_loss += loss.item()
        if i % 100 == 99:  # 每100个batch打印一次loss
            print(f'Epoch {epoch + 1}, Batch {i + 1}, Loss: {running_loss / 100:.3f}')
            running_loss = 0.0

print('Finished Training')
torch.save(model.state_dict(), 'chess_ai_model.pth')


AssertionError: Torch not compiled with CUDA enabled

In [18]:
from ai import DeepLearningAI

# 加载深度学习AI
deep_learning_ai = DeepLearningAI('chess_ai_model.pth')

# 与其他AI算法进行对战
# ai_battle(deep_learning_ai, minimax_ai, display=True)
ai_battle(deep_learning_ai, mcts_ai_0, display=True)


游戏开始！
蓝方AI: DeepLearningAI
红方AI: MCTSAI

[
  [[2, 2], [1, 1], [0, 0], [0, 0], [0, 0]],
  [[1, 1], [0, 0], [0, 0], [0, 0], [0, 0]],
  [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0]],
  [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0]],
  [[0, 0], [0, 0], [0, 0], [0, 0], [0, 0]]
]
第1步: 蓝方 落子在 (0, 0)
获胜概率: 0.00%
分数: 1.5
用时: 0.03s

[
  [[ 2,  2], [ 1,  1], [ 0,  0], [ 0,  0], [ 0,  0]],
  [[ 0,  2], [-2,  2], [-1,  1], [ 0,  0], [ 0,  0]],
  [[ 0,  0], [-1,  1], [ 0,  0], [ 0,  0], [ 0,  0]],
  [[ 0,  0], [ 0,  0], [ 0,  0], [ 0,  0], [ 0,  0]],
  [[ 0,  0], [ 0,  0], [ 0,  0], [ 0,  0], [ 0,  0]]
]
第2步: 红方 落子在 (1, 1)
获胜概率: 68.00%
分数: -3.5
用时: 2.08s



AssertionError: 须在值为0处落子, (0,0)为2